In [7]:
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import itertools as it
import math as math
from scipy.special import comb
import numpy as np

import sys, os, time, numpy as np, matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap, LinearSegmentedColormap
# from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
import numpy as np, matplotlib.pyplot as plt, matplotlib.colors,  matplotlib.cm as cm, os, sys
from mpl_toolkits.axes_grid1.axes_divider import make_axes_locatable
from mpl_toolkits.mplot3d import axes3d
import mpl_toolkits.mplot3d.art3d as art3d

home = os.path.abspath( os.getcwd() + "/.." )
sys.path.append(home)
from sklearn.cluster import KMeans

import numpy as np
import matplotlib.pyplot as plt
import cmath
import math
import pandas as pd

In [8]:
def transfer_matrix_method(a_x, a_y, m, n, t, w, f):
  k0 = (2*cmath.pi*f)/343 #wavenumber in air
  k0 = k0*1
  ds = []
  ls = []
  for i in range(m+n):
    d = (a_x/(m+n-1)) - ((m+n)/(m+n-1))*w
    l = a_y-2*t-d
    ls.append(l)
    ds.append(d)

  d_in = ds[0]
  l_in = ls[0]
  d_out = ds[-1]

  #print(l_in)
  #print(d_out)
  
  Ms = []

  for ii in range(len(ds)-1):
    summation = 0
    for i in range(10): #should be infinity

      k_yn_dash = (i*cmath.pi)/(ls[ii]+ds[ii]+(2*t))
      k_xn_dash = (k0**2 - k_yn_dash**2)**(1/2)
      #k_xn_dash = k_xn_dash*1.1
      #print(k_xn_dash)

      if i==0:
        psi_n_dash_dash = 1

      else:
        psi_n_dash_dash = ((2**(1/2))/(ds[ii]*k_yn_dash))*((cmath.sin(k_yn_dash*(t+ls[ii]+ds[ii])))-(cmath.sin(k_yn_dash*(t+ls[ii]))))
      summation += ((ds[ii]*k0)*(psi_n_dash_dash**2))/(a_y*k_xn_dash)

    delta1 = 1 + summation
    delta2 = 1 - summation

    alpha = 0
    beta = 0

    for i in range(10): #should be infinity
      k_yn = (i * cmath.pi)/(ls[ii]+ds[ii])
      k_xn = (k0**2 - k_yn**2)**(1/2)
      #k_xn = k_xn*1.1

      if i==0:
        phi_n1 = 1
        phi_n2 = 1

      else:
        phi_n1 = ((2)**(1/2)/ds[ii])*(cmath.sin(k_yn*ds[ii])/k_yn)
        phi_n2 = ((2)**(1/2)/(ds[ii]*k_yn))*((cmath.sin(k_yn*(ls[ii]+ds[ii])))-cmath.sin(k_yn*ls[ii]))
        
      alpha += ((k0*ds[ii])*(1+cmath.exp(2j*k_xn*ds[ii]))*phi_n1**2)/(k_xn*(ls[ii]+ds[ii])*(1-cmath.exp(2j*k_xn*ds[ii])))
      beta += (k0*ds[ii]*2*cmath.exp(1j*k_xn*ds[ii])*phi_n1*phi_n2)/(k_xn*(ls[ii]+ds[ii])*(1-cmath.exp(2j*k_xn*ds[ii])))

    M1_11 = (((alpha+1)**2-beta**2)*cmath.exp(-1j*k0*w))/(2*beta)
    M1_12 = (1-alpha**2+beta**2)/(2*beta)
    M1_21 = ((alpha**2)-1-(beta**2))/(2*beta)
    M1_22 = (((beta**2)-((alpha-1)**2))*cmath.exp(1j*k0*w))/(2*beta)
    M1 = [[M1_11, M1_12], [M1_21, M1_22]]
    Ms.append(M1)

  summation1 = 0
  for i in range(10):
    k_yn_dash = (i*cmath.pi)/(l_in+d_in+(2*t))
    k_xn_dash = (k0**2 - k_yn_dash**2)**(0.5)
    #k_xn_dash = k_xn_dash*1.1

    if i==0:
      #psi_n_dash = (1/d)*(((2-delta0n)**(1/2)*(t+d))-((2-delta0n)**(1/2)*(t))) #THIS
      psi_n_dash = 1
    else:
      #psi_n_dash = (1/d) * (((2-delta0n)**(1/2)*cmath.sin(k_yn_dash*(t+d))/k_yn_dash)-((2-delta0n)**(1/2)*cmath.sin(k_yn_dash*(t))/k_yn_dash))
      psi_n_dash = ((2)**(1/2)/d_in)*((cmath.sin(k_yn_dash*(t+d_in))/k_yn_dash)-(cmath.sin(k_yn_dash*(t))/k_yn_dash))

    summation1 += (k0*psi_n_dash**2)/(k_xn_dash)

  alpha_dash = (a_y/d_in) - summation1
  beta_dash = (a_y/d_in) + summation1

  M2_11 = (1-(beta_dash/2)) * cmath.exp(-1j*k0*w)
  M2_12 = (beta_dash/2) * cmath.exp(-1j*k0*w)
  M2_21 = 1 + (alpha_dash/2)
  M2_22 = -alpha_dash/2

  M2 = [[M2_11, M2_12], [M2_21, M2_22]]

  N = Ms[-1]
  for i in range(len(Ms)-1):
    N = np.matmul(N, Ms[len(Ms)-1-i])
  
  N = np.matmul(N, M2)

  #N = np.matmul(np.matmul(np.matmul(Ms[-1], Ms[-2]), Ms[-3]), M2)

  n11 = N[0][0]
  n12 = N[0][1]
  n21 = N[1][0]
  n22 = N[1][1]

  T = (2*d_out*((n21*n12)-(n11*n22)))/(a_y*((n12*delta1*cmath.exp(-1j*k0*w))+(n22*delta2)))
  R = -((n11*delta1)+(n21*delta2*cmath.exp(1j*k0*w)))/((n12*delta1)+(n22*delta2*cmath.exp(1j*k0*w)))
  
  magnitude = np.abs(T)
  phase = np.angle(T)
  #if phase<0:
  #  phase = phase+2*np.pi


  magnitude_reflection = np.abs(R) #10 * math.log10(np.abs(R))
  phase_reflection = np.angle(R)
  if phase_reflection<0:
    phase_reflection = phase_reflection+2*np.pi

  return magnitude, phase, magnitude_reflection, phase_reflection

In [9]:
wavelength_value = 8.661*10**(-3)
a_x = wavelength_value 
a_y = wavelength_value*0.7 
t = 0.0007810949749 
m = 1 #number of bars
n = 1 #number of bars
w = 0.002652927136
f = 40000

T, p, _, _ = transfer_matrix_method(a_x, a_y, m, n, t, w, f)
if p<0:
  p = ((p/np.pi)%2)
else:
  p = p /np.pi
print('Amplitude of transmission coefficient:', T)
print('Phase of transmission coefficient:', p)

Amplitude of transmission coefficient: 0.7326065018785177
Phase of transmission coefficient: 1.996010933882756
